# Offline Learning Resources for Comprehensive Project

## Load libraries and configure environment

In [1]:
%load_ext autoreload
%autoreload 2

import yaml
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from data.imagenet_loader import ContinualImageNet
from models.backbone import get_backbone
from strategies.joint_training import JointTrainer
from utils.metrics import CLMetrics

with open('config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

device = torch.device(cfg['experiment']['device'] if torch.cuda.is_available() else "cpu")
print(f"Config loaded. Device: {device}")

/home/dikhang_hcmut/miniconda3/envs/pytorch_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Config loaded. Device: cuda


## Load dataset

In [5]:
# Setup Data & Metrics
cl_data = ContinualImageNet(
    root_dir=cfg['data']['root_dir'],
    num_tasks=cfg['data']['num_tasks'],
    total_classes=cfg['data']['total_classes'],
    batch_size=cfg['training']['batch_size'],
    num_workers=cfg['data']['num_workers']
)

cl_metrics = CLMetrics(cfg['data']['num_tasks'])
history = {'acc': [], 'forget': []}

model = get_backbone(cfg)
trainer = JointTrainer(model, device, cfg)

FileNotFoundError: [Errno 2] No such file or directory: './datasets/imagenet256/train'

## Train model with offline learning approach

In [2]:
num_tasks = cfg['data']['num_tasks']
epochs = cfg['training']['epochs_per_task']

### Training loop

In [3]:
for task_id in range(num_tasks):
    print(f"\n{'='*10} PHASE {task_id+1}/{num_tasks} {'='*10}")
    
    train_loader = cl_data.get_data_loader(task_id, mode='train', cumulative=True)
    trainer.train(train_loader, epochs=epochs)

    print("Evaluating metrics...")
    for test_task in range(task_id + 1):
        val_loader = cl_data.get_data_loader(test_task, mode='val', cumulative=False)
        acc = trainer.evaluate(val_loader)
        cl_metrics.update(train_phase=task_id, test_task=test_task, accuracy=acc)
        print(f" > Task {test_task} Accuracy: {acc:.2f}%")
        
    avg_acc, avg_forget = cl_metrics.calculate_metrics(task_id)
    history['acc'].append(avg_acc)
    history['forget'].append(avg_forget)
    
    print(f"Avg Accuracy: {avg_acc:.2f}% | Avg Forgetting: {avg_forget:.2f}%")


========== PHASE 1/5 ==========


NameError: name 'cl_data' is not defined